# Capítulo 07 — Instruction Tuning: Criando um Assistente

🎯 **Objetivos:** Transformar o modelo completador em um assistente útil usando **SFT (Supervised Fine-Tuning)**.

![SFT](./infograficos/04-pipeline-sft.png)

In [ ]:
# ============================================================
# Setup do repositório no Colab
# ============================================================
import os, sys
REPO_NAME = "fazendo-um-llm-do-zero"
if 'google.colab' in str(get_ipython()):
    if not os.path.exists(REPO_NAME):
        get_ipython().system(f"git clone https://github.com/vongrossi/{REPO_NAME}.git")
    if os.path.exists(REPO_NAME) and os.getcwd().split('/')[-1] != REPO_NAME:
        os.chdir(REPO_NAME)
if os.getcwd() not in sys.path: sys.path.append(os.getcwd())
print("📂 Diretório atual:", os.getcwd())

In [ ]:
import os, sys, torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from lib.gptmini import GPTConfig, GPTMini

device = "cuda" if torch.cuda.is_available() else "cpu"

# 📂 Carregamento do Checkpoint do Cap 05
if not os.path.exists("gpt_checkpoint.pt"):
    from google.colab import files
    print("📤 Por favor, suba o 'gpt_checkpoint.pt' gerado no Capítulo 05:")
    uploaded = files.upload()

ckpt = torch.load("gpt_checkpoint.pt", map_location=device, weights_only=False)
stoi, itos = ckpt['stoi'], ckpt['itos']
config = ckpt['config']
context_size = config.context_size # Importante: manter o mesmo do pre-treino

encode = lambda s: [stoi[c] for c in s if c in stoi]
decode = lambda l: ''.join([itos[i] for i in l])

print(f"🧠 Modelo pré-treinado carregado!")
print(f"📏 Janela de contexto: {context_size} | Vocabulário: {len(stoi)}")

## 1. Dataset de Instruções

Ensinamos o modelo que a estrutura `### Comando:` pede uma resposta em `### Resposta:`.
**Nota:** Agora usamos o `context_size` sincronizado com o modelo ({context_size}).

In [ ]:
instructions = [
    {"q": "o que o gato fez?", "a": "o gato subiu no telhado e pulou o muro."},
    {"q": "onde o cachorro dormiu?", "a": "o cachorro dormiu no tapete do sofa."},
    {"q": "defina inteligencia artificial", "a": "artificial inteligencia e o estudo de algoritmos."},
    {"q": "o que e machine learning?", "a": "machine learning permite que sistemas aprendam padroes."}
]

def build_sft_dataset(data, context_size):
    X, Y, masks = [], [], []
    for item in data:
        cmd = f"### Comando:\n{item['q']}\n\n### Resposta:\n"
        full = cmd + item['a']
        
        ids = encode(full)
        cmd_tokens = encode(cmd)
        cmd_len = len(cmd_tokens)
        
        for i in range(len(ids) - context_size):
            x_batch = ids[i : i+context_size]
            y_batch = ids[i+1 : i+context_size+1]
            
            # Lógica da Máscara: 0 no que for comando, 1 no que for resposta
            # Precisamos calcular onde o comando termina dentro dessa janela x_batch
            m = []
            for j in range(i, i + context_size):
                if j < cmd_len: m.append(0)
                else: m.append(1)
            
            X.append(x_batch)
            Y.append(y_batch)
            masks.append(m)
            
    return torch.tensor(X).to(device), torch.tensor(Y).to(device), torch.tensor(masks).to(device)

X, Y, M = build_sft_dataset(instructions, context_size)
print(f"📦 Amostras de Alinhamento: {len(X)}")

## 2. Treinamento com Máscara de Loss

Otimizamos apenas a geração da resposta, ignorando o comando na hora de calcular o erro.

![Masking](./infograficos/03-mascaramento-loss-resposta.png)

In [ ]:
model = GPTMini(config).to(device)
model.load_state_dict(ckpt['state_dict'])
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)

loss_history = []
model.train()
print("🔨 Alinhando o assistente...")
for step in range(601):
    idx = torch.randint(len(X), (8,))
    xb, yb, mb = X[idx], Y[idx], M[idx]
    
    logits, _ = model(xb)
    
    # Cálculo da Loss Mascarada
    B, T, V = logits.shape
    loss = F.cross_entropy(logits.view(-1, V), yb.view(-1), reduction='none')
    loss = (loss * mb.view(-1)).mean()
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    loss_history.append(loss.item())
    if step % 200 == 0: print(f"Step {step:03d} | Loss {loss.item():.4f}")

plt.figure(figsize=(8, 3))
plt.plot(loss_history, color='#34A853')
plt.title("Curva de Alinhamento (SFT)")
plt.show()

## 3. Teste do Assistente Alinhado

O modelo agora deve responder seguindo o protocolo de comando.

In [ ]:
@torch.no_grad()
def ask(model, question):
    model.eval()
    prompt = f"### Comando:\n{question.lower()}\n\n### Resposta:\n"
    idx = torch.tensor(encode(prompt)).unsqueeze(0).to(device)
    
    for _ in range(60):
        # GARANTIA: Nunca ultrapassar o contexto original do modelo
        idx_cond = idx[:, -context_size:]
        logits, _ = model(idx_cond)
        
        # Escolha determinística (Greedy)
        next_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        idx = torch.cat([idx, next_id], dim=1)
        
        if itos[next_id.item()] == ".": break
        
    return decode(idx[0].tolist())

print("🤖 TESTE DE INTERAÇÃO:")
print("-" * 30)
print(ask(model, "o que o gato fez?"))
print(ask(model, "o que e machine learning?"))

## 🏁 Conclusão da Jornada

Você completou a série! 

Transformou um modelo estatístico em um assistente capaz de seguir intenções humanas. Este é o fundamento do alinhamento de IA.

![Avaliação](./infograficos/05-avaliacao-respostas.png)